In [2]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, String, Float, Integer
from sqlalchemy import create_engine
from sqlalchemy.orm import Session

import sqlite3

## Offstreertcarpark Database

In [3]:
# Connect to offstreetparking database
conn = sqlite3.connect('./Resources/offstreetcarpark.db')

In [4]:
# Show the tables in the database
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('parking',)]


In [16]:
# Show first 5 rows of the table
cursor.execute("SELECT * FROM parking LIMIT 5;")
print(cursor.fetchall())

[(0, 2011, 912, 102432, 102432, '35 Degraves Street PARKVILLE 3052', 'Parkville', 'Residential', 2, 144.9568, -37.79384), (1, 2011, 912, 102433, 102433, '37 Degraves Street PARKVILLE 3052', 'Parkville', 'Residential', 2, 144.95673, -37.79384), (2, 2011, 912, 107328, 107328, '110 Park Drive PARKVILLE 3052', 'Parkville', 'Residential', 1, 144.95644, -37.793929999999996), (3, 2011, 913, 106769, 106769, '58 Morrah Street PARKVILLE 3052', 'Parkville', 'Residential', 1, 144.9555, -37.79525), (4, 2011, 913, 107340, 107340, '54-58 Park Drive PARKVILLE 3052', 'Parkville', 'Residential', 2, 144.95533, -37.79513)]


In [6]:
# Drop the table if it already exists
cursor.execute("DROP TABLE IF EXISTS parking;")
conn.commit()

In [7]:
# Create new parking table
cursor.execute("CREATE TABLE parking(\
            id INT PRIMARY KEY  NOT NULL,\
            census_year INT,\
            block_id INT,\
            property_id INT,\
            base_property_id INT,\
            building_address CHAR(50),\
            clue_small_area CHAR(20),\
            parking_type CHAR(20),\
            parking_spaces INT,\
            longitude FLOAT,\
            latitude FLOAT\
            );")
conn.commit()             

In [8]:
Base = declarative_base()
database_path = "./Resources/offstreetcarpark.db"
engine = create_engine(f"sqlite:///{database_path}")
Base.metadata.create_all(engine)

In [9]:
session = Session(bind=engine)
session

In [10]:

class Park(Base):
    __tablename__ = 'parking'
    id = Column(Integer(), primary_key=True)
    census_year = Column(Integer)
    block_id = Column(Integer)
    property_id = Column(Integer)
    base_property_id = Column(Integer)
    building_address = Column(String)
    clue_small_area = Column(String)
    parking_type = Column(String)
    parking_spaces = Column(Integer)
    longitude = Column(Float)
    latitude = Column(Float)

In [11]:
carpark = session.query(Park)

In [15]:
# Import csv file to DataFrame
import pandas as pd
carpark_df = pd.read_csv("./Resources/tbl_off_street_car_park.csv")

# Add id column
carpark_df['id'] = carpark_df.index

# Load data into parking table
carpark_df.to_sql(name='parking', con=engine, if_exists='append', index=False)

carpark_df.head()

,census_year,block_id,property_id,base_property_id,building_address,clue_small_area,parking_type,parking_spaces,longitude,latitude,id
0,2011,912,102432,102432,35 Degraves Street PARKVILLE 3052,Parkville,Residential,2,144.95680,-37.79384,0
1,2011,912,102433,102433,37 Degraves Street PARKVILLE 3052,Parkville,Residential,2,144.95673,-37.79384,1
2,2011,912,107328,107328,110 Park Drive PARKVILLE 3052,Parkville,Residential,1,144.95644,-37.79393,2
3,2011,913,106769,106769,58 Morrah Street PARKVILLE 3052,Parkville,Residential,1,144.95550,-37.79525,3
4,2011,913,107340,107340,54-58 Park Drive PARKVILLE 3052,Parkville,Residential,2,144.95533,-37.79513,4


In [18]:
carpark_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129657 entries, 0 to 129656
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   census_year       129657 non-null  int64  
 1   block_id          129657 non-null  int64  
 2   property_id       129657 non-null  int64  
 3   base_property_id  129657 non-null  int64  
 4   building_address  129657 non-null  object 
 5   clue_small_area   129657 non-null  object 
 6   parking_type      129657 non-null  object 
 7   parking_spaces    129657 non-null  int64  
 8   longitude         128573 non-null  float64
 9   latitude          128573 non-null  float64
 10  id                129657 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 10.9+ MB


In [17]:
# Show first 5 rows of the table
cursor.execute("SELECT * FROM parking LIMIT 5;")
print(cursor.fetchall())

[(0, 2011, 912, 102432, 102432, '35 Degraves Street PARKVILLE 3052', 'Parkville', 'Residential', 2, 144.9568, -37.79384), (1, 2011, 912, 102433, 102433, '37 Degraves Street PARKVILLE 3052', 'Parkville', 'Residential', 2, 144.95673, -37.79384), (2, 2011, 912, 107328, 107328, '110 Park Drive PARKVILLE 3052', 'Parkville', 'Residential', 1, 144.95644, -37.793929999999996), (3, 2011, 913, 106769, 106769, '58 Morrah Street PARKVILLE 3052', 'Parkville', 'Residential', 1, 144.9555, -37.79525), (4, 2011, 913, 107340, 107340, '54-58 Park Drive PARKVILLE 3052', 'Parkville', 'Residential', 2, 144.95533, -37.79513)]


In [19]:
session.close()